In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from llama_index.core import Settings

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

# ollama
Settings.llm = Ollama(model="deepseek-r1:7b", request_timeout=360.0)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.
INFO:datasets:PyTorch version 2.2.2 available.
PyTorch version 2.2.2 available.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
Load pretrained SentenceTransformer: BAAI/bge-m3
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [ ]:
!export OPENAI_API_KEY=sk-S1h0hPEa3Qa5pCAJHNf6sd9EHxjGSlzd99xOgE4GYRpdBxek
!export OPENAI_BASE_URL=https://api.chatanywhere.tech

## Loading using SimpleDirectoryReader

In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("../../data").load_data()

## Using Readers from LlamaHub

In [ ]:
import os
from llama_index.core import download_loader
from llama_index.readers.database import DatabaseReader

reader = DatabaseReader(
    scheme=os.getenv("DB_SCHEME"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASS"),
    dbname=os.getenv("DB_NAME"),
)

query = "SELECT * FROM users"
documents = reader.load_data(query=query)

## Creating Documents directly

In [ ]:
from llama_index.core import Document

doc = Document(text="nothing to commit, working tree clean")

## Transformations
### High-Level Transformation API

In [5]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)
vector_index.as_query_engine()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex

# global
from llama_index.core import Settings
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)

Settings.text_splitter = text_splitter

# per-index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter]
)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "Write an email to the user given their background information."
)
print(response)